# 评估simple rag中的块大小
选择合适的块大小对rag管道检索的准确性至关重要。
---
以下方式评估不同的块大小:
- 从 PDF 中提取文本
- 将文本分割成不同大小的块
- 为每个块创建嵌入
- 为查询检索相关块
- 使用检索到的块生成响应
- 评估响应质量
- 比较不同块大小的结果
---
实现步骤：
- 提取文本：按页面
- 分割成大小不同的块，并创建嵌入
- 根据查询检索相关块
- 使用检索到的文本块用模型生成回答
- 评估不同大小快的检索回答质量

In [1]:
import fitz
import os
import numpy as np
import google.generativeai as genai
from dotenv import load_dotenv
import json

try:
    load_dotenv()
    print("环境已配置")
except:
    print("检查环境文件是否已配置")

环境已配置


In [2]:
genai.configure(
    api_key = os.getenv("API_KEY"),
    transport="rest"
)

In [9]:
def extra_text_from_pdf(pdf_path):
    mypdf = fitz.open(pdf_path)
    all_text = ""

    for page in mypdf:
        all_text += page.get_text("text") + " "

    return all_text.strip()

pdf_path = "../data/AI_Information.en.zh-CN.pdf"
extra_text = extra_text_from_pdf(pdf_path)
print(extra_text[:500])

理解⼈⼯智能
第⼀章：⼈⼯智能简介
⼈⼯智能 (AI) 是指数字计算机或计算机控制的机器⼈执⾏通常与智能⽣物相关的任务的能⼒。该术
语通常⽤于开发具有⼈类特有的智⼒过程的系统，例如推理、发现意义、概括或从过往经验中学习
的能⼒。在过去的⼏⼗年中，计算能⼒和数据可⽤性的进步显著加速了⼈⼯智能的开发和部署。
历史背景
⼈⼯智能的概念已存在数个世纪，经常出现在神话和⼩说中。然⽽，⼈⼯智能研究的正式领域始于
20世纪中叶。1956年的达特茅斯研讨会被⼴泛认为是⼈⼯智能的发源地。早期的⼈⼯智能研究侧
重于问题解决和符号⽅法。20世纪80年代专家系统兴起，⽽20世纪90年代和21世纪初，机器学习
和神经⽹络取得了进步。深度学习的最新突破彻底改变了这⼀领域。
现代观察
现代⼈⼯智能系统在⽇常⽣活中⽇益普及。从 Siri 和 Alexa 等虚拟助⼿，到流媒体服务和社交媒体
上的推荐算法，⼈⼯智能正在影响我们的⽣活、⼯作和互动⽅式。⾃动驾驶汽⻋、先进的医疗诊断
技术以及复杂的⾦融建模⼯具的发展，彰显了⼈⼯智能应⽤的⼴泛性和持续增⻓。此外，⼈们对其
伦理影响、偏⻅和失业的担忧也⽇益凸显。
第⼆章：⼈⼯智能


# 对提取的文本块进行分块

In [11]:
def chunk_text(text, n, overlap):
    """
    将文本分割为重叠的块
    参数：
    text（str）：要分割的文本
    n(int):每个块的字符数
    overlap(int):需要重叠的字符数

    return：
    List[str]:文本块列表
    """
    chunks = []
    for i in range(0, len(text), n-overlap):
        try:
            chunks.append(text[i:i+n])
        except:
            chunks.append(text[i:])

    return chunks

# 定义需要评估的不同块
chunk_sizes = [128, 256, 512]

# 创建字典存储每个块大小对应的文本块
text_chunks_dict = {size:chunk_text(extra_text, size, size//5) for size in chunk_sizes}
# print(text_chunks_dict)
# 打印每个块大小生成的块数量
for size, chunks in text_chunks_dict.items():
    print(f"chunk_size:{size}, Number of chunks:{len(chunks)}")
    

chunk_size:128, Number of chunks:98
chunk_size:256, Number of chunks:50
chunk_size:512, Number of chunks:25


# 为文本片段创建嵌入

In [13]:
from tqdm import tqdm
import numpy as np
import os
# 假设client已经被正确初始化和配置

In [17]:
def create_embeddings(texts):
    """
    为文本列表生成嵌入

    Args:
    texts (List[str]): 输入文本列表.

    Returns:
    List[np.ndarray]: List of numerical embeddings.
    """
    # 确保每次调用不超过64条文本
    batch_size = 64
    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = genai.embed_content(
        model = os.getenv("EMBEDDING_MODEL"),
        content = texts,
        task_type = "RETRIEVAL_DOCUMENT"
    )
        # 将响应转换为numpy数组列表并添加到embeddings列表中
        embeddings.extend([np.array(embedding.embedding) for embedding in response.data])

    return embeddings

# 假设text_chunks_dict是一个字典，键是块大小，值是文本块列表
chunk_embeddings_dict = {}
for size, chunks in tqdm(text_chunks_dict.items(), desc="Generating Embeddings"):
    chunk_embeddings_dict[size] = create_embeddings(chunks)

Generating Embeddings:   0%|                                                                     | 0/3 [00:04<?, ?it/s]


AttributeError: 'dict' object has no attribute 'data'

In [19]:
def cosine_similarity(vec1, vec2):
    """
    Computes cosine similarity between two vectors.

    Args:
    vec1 (np.ndarray): First vector.
    vec2 (np.ndarray): Second vector.

    Returns:
    float: Cosine similarity score.
    """

    # Compute the dot product of the two vectors
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [20]:
def retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, k=5):
    """
    检索与查询最相关的前k个文本块

    Args:
    query (str): 用户查询
    text_chunks (List[str]): 文本块列表
    chunk_embeddings (List[np.ndarray]): 文本块的嵌入列表
    k (int): 返回的前k个块数量

    Returns:
    List[str]: 最相关的文本块列表
    """
    # 为查询生成一个嵌入 - 将查询作为列表传递并获取第一个项目
    query_embedding = create_embeddings([query])[0]

    # 计算查询嵌入与每个块嵌入之间的余弦相似度
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]

    # 获取前k个最相似块的索引
    top_indices = np.argsort(similarities)[-k:][::-1]

    # 返回前k个最相关的文本块
    return [text_chunks[i] for i in top_indices]

In [21]:
# 从 JSON 文件加载验证数据
with open('../data/val.json', encoding="utf-8") as f:
    data = json.load(f)

# 从验证数据中提取第一个查询
query = data[3]['question']

# 对于每个块大小，检索相关的文本块
retrieved_chunks_dict = {size: retrieve_relevant_chunks(query, text_chunks_dict[size], chunk_embeddings_dict[size]) for size in chunk_sizes}

# 打印块大小为 256 的检索到的文本块
print(retrieved_chunks_dict[256])

KeyError: 128

# 基于检索到的片段生成响应

In [22]:
# AI 助手的系统提示
system_prompt = "你是一个AI助手，严格根据给定的上下文进行回答。如果无法直接从提供的上下文中得出答案，请回复：'我没有足够的信息来回答这个问题。'"

def generate_response(query, system_prompt, retrieved_chunks):
    """
    基于检索到的文本块生成 AI 回答。

    Args:
    query (str): 用户查询
    retrieved_chunks (List[str]): 检索到的文本块列表
    model (str): AI model.

    Returns:
    str: AI-generated response.
    """
    # 将检索到的文本块合并为一个上下文字符串
    context = "\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])

    # 通过组合上下文和查询创建用户提示
    user_prompt = f"{context}\n\nQuestion: {query}"

    # Generate the AI response using the specified model
    response = client.chat.completions.create(
        model=os.getenv("LLM_MODEL_ID"),
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # Return the content of the AI response
    return response.choices[0].message.content

# 为每个块大小生成 AI 回答
ai_responses_dict = {size: generate_response(query, system_prompt, retrieved_chunks_dict[size]) for size in chunk_sizes}

# 打印块大小为 256 的回答
print(ai_responses_dict[256])

NameError: name 'retrieved_chunks_dict' is not defined

In [23]:
# 定义评估评分系统的常量
SCORE_FULL = 1.0     # 完全匹配或完全令人满意
SCORE_PARTIAL = 0.5  # 部分匹配或部分令人满意
SCORE_NONE = 0.0     # 无匹配或不令人满意

In [24]:
# 定义严格的评估提示模板
FAITHFULNESS_PROMPT_TEMPLATE = """
评估 AI 回答与真实答案的一致性、忠实度。
用户查询: {question}
AI 回答: {response}
真实答案: {true_answer}

一致性衡量 AI 回答与真实答案中的事实对齐的程度，且不包含幻觉信息。
忠实度衡量的是AI的回答在没有幻觉的情况下与真实答案中的事实保持一致的程度。

指示：
- 严格使用以下值进行评分：
    * {full} = 完全一致，与真实答案无矛盾
    * {partial} = 部分一致，存在轻微矛盾
    * {none} = 不一致，存在重大矛盾或幻觉信息
- 仅返回数值评分（{full}, {partial}, 或 {none}），无需解释或其他附加文本。
"""

In [25]:
RELEVANCY_PROMPT_TEMPLATE = """
评估 AI 回答与用户查询的相关性。
用户查询: {question}
AI 回答: {response}

相关性衡量回答在多大程度上解决了用户的问题。

指示：
- 严格使用以下值进行评分：
    * {full} = 完全相关，直接解决查询
    * {partial} = 部分相关，解决了一些方面
    * {none} = 不相关，未能解决查询
- 仅返回数值评分（{full}, {partial}, 或 {none}），无需解释或其他附加文本。
"""

In [26]:
def evaluate_response(question, response, true_answer):
        """
        根据忠实度和相关性评估 AI 生成的回答质量

        Args:
        question (str): 用户的原始问题
        response (str): 被评估的 AI 生成的回答
        true_answer (str): 作为基准的真实答案

        Returns:
        Tuple[float, float]: 包含 (忠实度评分, 相关性评分) 的元组。
                             每个评分可能是：1.0（完全匹配）、0.5（部分匹配）或 0.0（无匹配）。
        """
        # 格式化评估提示
        faithfulness_prompt = FAITHFULNESS_PROMPT_TEMPLATE.format(
                question=question,
                response=response,
                true_answer=true_answer,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )

        relevancy_prompt = RELEVANCY_PROMPT_TEMPLATE.format(
                question=question,
                response=response,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )

        # 模型进行忠实度评估
        faithfulness_response = client.chat.completions.create(
               model=os.getenv("LLM_MODEL_ID"),
                temperature=0,
                messages=[
                        {"role": "system", "content": "你是一个客观的评估者，仅返回数值评分。"},
                        {"role": "user", "content": faithfulness_prompt}
                ]
        )

        # 模型进行相关性评估
        relevancy_response = client.chat.completions.create(
                model=os.getenv("LLM_MODEL_ID"),
                temperature=0,
                messages=[
                        {"role": "system", "content": "你是一个客观的评估者，仅返回数值评分。"},
                        {"role": "user", "content": relevancy_prompt}
                ]
        )

        # 提取评分并处理潜在的解析错误
        try:
                faithfulness_score = float(faithfulness_response.choices[0].message.content.strip())
        except ValueError:
                print("Warning: 无法解析忠实度评分，将默认为 0")
                faithfulness_score = 0.0

        try:
                relevancy_score = float(relevancy_response.choices[0].message.content.strip())
        except ValueError:
                print("Warning: 无法解析相关性评分，将默认为 0")
                relevancy_score = 0.0

        return faithfulness_score, relevancy_score

# 第一条验证数据的真实答案
true_answer = data[3]['ideal_answer']

# 评估块大小为 256 和 128 的回答
faithfulness, relevancy = evaluate_response(query, ai_responses_dict[256], true_answer)
faithfulness2, relevancy2 = evaluate_response(query, ai_responses_dict[128], true_answer)

# 打印评估分数
print(f"忠实度评分 (Chunk Size 256): {faithfulness}")
print(f"相关性评分 (Chunk Size 256): {relevancy}")

print(f"\n")

print(f"忠实度评分 (Chunk Size 128): {faithfulness2}")
print(f"忠实度评分 (Chunk Size 128): {relevancy2}")

NameError: name 'ai_responses_dict' is not defined

# 小结
本次的分块其实类似于穷举，将几个分块的数量进行对比得到最佳答案。
- 重点：最后的评估提示词规范结构化提问和回答需要学习